In [3]:
from tesi.database.di import get_session_maker
from tesi.zappai.di import get_cds_api, get_crop_repository, get_crop_yield_data_repository, get_location_repository
from tesi.zappai.exceptions import CropNotFoundError
from tesi.zappai.repositories.climate_generative_model_repository import FEATURES as CLIMATE_GENERATIVE_MODEL_FEATURES
from tesi.zappai.repositories.dtos import CropYieldDataDTO

session_maker = get_session_maker()
location_repository = get_location_repository(session_maker=session_maker)
crop_repository = get_crop_repository(session_maker=session_maker)
crop_yield_data_repository = get_crop_yield_data_repository(
    session_maker=session_maker,
    crop_repository=crop_repository,
    location_repository=location_repository
)

crop = await crop_repository.get_crop_by_name("maize")
if crop is None:
    raise CropNotFoundError()
crop_yield_data = await crop_yield_data_repository.get_crop_yield_data(crop_id=crop.id)
crop_yield_data_df = CropYieldDataDTO.from_list_to_dataframe(crop_yield_data)
crop_yield_data_df


,index,id,location_id,crop_id,sowing_year,sowing_month,harvest_year,harvest_month,_yield
0,707,622ba95e-ea84-474f-aa55-44ccf455f9f7,3d3b83c1-9dd2-4b5b-a06e-bd1f83a8188c,ab7e4a12-5fe5-4701-8cb0-e2f6d084c0f3,1980,3,1980,7,2400.0
1,708,73c00f2e-a832-4ee5-b366-787eed7fd344,3d3b83c1-9dd2-4b5b-a06e-bd1f83a8188c,ab7e4a12-5fe5-4701-8cb0-e2f6d084c0f3,1980,3,1980,7,2500.0
2,1056,471d6b9b-44a8-4afd-85d1-e9214fb06864,3d3b83c1-9dd2-4b5b-a06e-bd1f83a8188c,ab7e4a12-5fe5-4701-8cb0-e2f6d084c0f3,1980,3,1980,7,3093.0
3,1057,8c9962d2-dd8a-4952-aa9e-8fd428134287,3d3b83c1-9dd2-4b5b-a06e-bd1f83a8188c,ab7e4a12-5fe5-4701-8cb0-e2f6d084c0f3,1980,3,1980,7,5678.0
4,1685,f13cb90d-8dfb-4352-8a23-b4c8a885482c,423761b0-10b5-4ffa-b56b-ef5df7215e85,ab7e4a12-5fe5-4701-8cb0-e2f6d084c0f3,1980,4,1980,10,4400.0
...,...,...,...,...,...,...,...,...,...
1685,75,203b5e10-f00a-4313-86bc-e3a20129a52e,d7636c4f-6399-4275-8fab-4d3f531d898a,ab7e4a12-5fe5-4701-8cb0-e2f6d084c0f3,2017,4,2017,9,4050.0
1686,97,4d82f9a2-6c27-4b8a-84f0-65a2f9dfdbeb,8d3b2bfc-0d0f-4ba2-b2aa-a30fdda829ff,ab7e4a12-5fe5-4701-8cb0-e2f6d084c0f3,2017,4,2017,8,10895.0
1687,103,e4c5a60e-bade-4e0e-bd6a-773293f2dba8,8d3b2bfc-0d0f-4ba2-b2aa-a30fdda829ff,ab7e4a12-5fe5-4701-8cb0-e2f6d084c0f3,2017,4,2017,8,12114.0
1688,809,504ab4d9-419a-4742-aa25-1316667215c8,e22382f9-1e49-4eb5-acad-9e7d45a9e788,ab7e4a12-5fe5-4701-8cb0-e2f6d084c0f3,2017,5,2017,9,9670.0


In [4]:
import pandas as pd
from tesi.zappai.di import (
    get_future_climate_data_repository,
    get_past_climate_data_repository,
)
from tesi.zappai.exceptions import LocationNotFoundError
from tesi.zappai.repositories.dtos import ClimateDataDTO

cds_api = get_cds_api()
past_climate_data_repository = get_past_climate_data_repository(
    session_maker=session_maker,
    cds_api=cds_api,
    location_repository=location_repository,
)
future_climate_data_repository = get_future_climate_data_repository(
    session_maker=session_maker, cds_api=cds_api
)

enriched_crop_yield_data = pd.DataFrame()

for index, row in crop_yield_data_df.iterrows():
    location = await location_repository.get_location_by_id(row["location_id"])
    if location is None:
        raise LocationNotFoundError()
    past_climate_data_df = ClimateDataDTO.from_list_to_dataframe(
        await past_climate_data_repository.get_past_climate_data(
            location_id=location.id,
            year_from=row["sowing_year"],
            month_from=row["sowing_month"],
            year_to=row["harvest_year"],
            month_to=row["harvest_month"],
        )
    )
    climate_data_stats = past_climate_data_df.agg(
        {
            feature: ["mean", "sum", "std", "min", "max"]
            for feature in CLIMATE_GENERATIVE_MODEL_FEATURES
        }, axis=1
    )
    enriched_crop_yield_data = pd.concat([enriched_crop_yield_data, row])
    break
climate_data_stats

KeyError: "Column(s) ['2m_dewpoint_temperature', '2m_temperature', 'soil_temperature_level_3', 'surface_net_solar_radiation', 'surface_net_thermal_radiation', 'surface_solar_radiation_downwards', 'surface_thermal_radiation_downwards', 'total_cloud_cover', 'total_precipitation', 'volumetric_soil_water_layer_3'] do not exist"